<a href="https://colab.research.google.com/github/igardner23/FrostedReports/blob/master/reporting_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Harrison:API(
        url="http://harrison.frostedleaf.co",
        consumer_key="ck_35492293d682ff14ffb5da8e0ba39a757048c2b6",
        consumer_secret="cs_2c647287461a14cd8d41354a85dabdef198bee3f",
        version="wc/v1"
    )


In [1]:
!pip install woocommerce


  Created wheel for woocommerce: filename=WooCommerce-2.1.1-cp36-none-any.whl size=6414 sha256=00cfcaf17ae792fb1963cc8d77f2189c59eac7767baf4b4a44a0c7961318b352
  Stored in directory: /root/.cache/pip/wheels/be/07/ed/97c52b6fc4760667ede55c791aa57804fbd296ebf53bcb86a1
  Created wheel for ordereddict: filename=ordereddict-1.1-cp36-none-any.whl size=3554 sha256=d6eb51382365b8d3ec035e6ea906c98a07a844aec2ce2b25143ff8264e3bd0b6
  Stored in directory: /root/.cache/pip/wheels/78/d3/a8/9f52c7a389a0ffba6f575a3886b5f7a55461fb6ec34aa5fd38
Successfully built woocommerce ordereddict


In [2]:

from woocommerce import API  
federal={'url':'http://www.frostedleaf.com/selfserve', 'consumerkey':'ck_19e902128d4171e9c304f6a4cd2b6a87d719a3e3', 'secretkey':'cs_0e3db2bc275be63d4365c3a9335ef71d0bc24f81'}  
wcapi = API(
        url=federal['url'],
        consumer_key=federal['consumerkey'],
        consumer_secret=federal['secretkey'],
        version="wc/v1"
    )

Import the WooCommerce API and initialize it with API Keys and sitename, v1 API at the moment, god need to throw this whole thing away.

In [3]:
TestBatch=wcapi.get('orders', params={'per_page':100}).json()

In [4]:
from datetime import datetime
datetime.strptime(TestBatch[0]['date_completed'], '%Y-%m-%dT%H:%M:%S')

datetime.datetime(2020, 11, 13, 6, 9, 5)

Pull first 100 orders ((Still have to add in my pagination function))

In [5]:
import pandas as pd
from datetime import datetime
df=pd.DataFrame(TestBatch)
df['date_completed']


0     2020-11-13T06:09:05
1     2020-11-12T22:02:46
2     2020-11-12T21:46:22
3     2020-11-12T21:44:48
4     2020-11-12T21:14:33
             ...         
95    2020-11-12T08:46:22
96    2020-11-12T16:42:35
97    2020-11-12T21:56:12
98    2020-11-12T12:23:16
99    2020-11-11T21:54:32
Name: date_completed, Length: 100, dtype: object

In [6]:
tester=[batch for batch in TestBatch if [x for x in batch['line_items'] if x['composite_parent']!='']]

For ease of access taking the initial 100 orders pulled and putting them into a dataframe. Everything looks good, now to create the actual report. In order to do this, I need to iterate through each order pulling the datetime of the order, and information from each line item of the order. Then I will need to take each line item and grab the variation and product information.

In [7]:
tester

[]

In [8]:
def GenerateProducts(thedate):
  ProductOrders=[]
  page=1
  ## First iterate through until we find a page that contains the date.
  while thedate not in [order['date_completed'].split('T')[0] for order in wcapi.get('orders', params={'per_page':100, 'page':page}).json()]:
    page+=1
    print('No orders detected, trying page number '+str(page))
  ## Then we want to repeat the product adding for each page that DOES contain the date
  while thedate in [order['date_completed'].split('T')[0] for order in wcapi.get('orders', params={'per_page':100, 'page':page}).json()]:
    orderbatch=wcapi.get('orders', params={'per_page':100, 'page':page}).json()
    ## Just defining that as a variable we're going to loop through, I probably could just get this a single time and write with alot less API calls
    for order in orderbatch:
      timestamp=datetime.strptime(order['date_completed'], '%Y-%m-%dT%H:%M:%S')
      ## Grab the timestamp from the order.
      for line_item in order['line_items']:
        if line_item['variation_id'] == 0:
          ProductOrders.append({'datetime':timestamp, 'lineitem':line_item, 'product_info':wcapi.get('products/'+str(line_item['product_id'])).json()})
        else:
          ProductOrders.append({'datetime':timestamp, 'lineitem':line_item, 'product_info':wcapi.get('products/'+str(line_item['variation_id'])).json()})
    return ProductOrders

    
    

def GenerateReport(ProductList):
    Report=[]
    Compositer={}
    for productinfo in ProductList:
        ## Sets the variable productinfo for each item in ProductList
        ## If you see this weird shit, just split it.
        if 'LeafCoin' in productinfo['product_info']['name']:
          continue
        if '&rarr;' in productinfo['product_info']['name']:
            productinfo['product_info']['name']=productinfo['product_info']['name'].split('&rarr;')[1].replace('-',' ')
        productinfo['product_info']['name']=productinfo['product_info']['name'].replace('-',' ')
        ## Then let's go ahead and set up that weight variable for use in Total Weight and Unit of Measure
        print(productinfo['product_info']['weight'], productinfo['product_info']['name'])
        if productinfo['product_info']['weight'] == '':
            ## Ugh, I have to be able to filter this better, maybe turning it into a function? Turning the
            ## For loop into something else?
            ## IDK
            productinfo['product_info']['weight']=1
            ## Reuse these things for the weight limiting even though that will probably not function...
            ## Ugh direly need to start implementing a number of things ughhh
        if productinfo['lineitem']['quantity'] > 1:
            weight=float(productinfo['product_info']['weight'])*productinfo['lineitem']['quantity']
        else:
            weight=float(productinfo['product_info']['weight'])
        if 'Flower' in [category['name'] for category in productinfo['product_info']['categories']]:
            typeweight='Grams'
        else:
            typeweight='Each'
        if len(productinfo['lineitem']['composite_children']) > 0:
          Compositer[str(productinfo['lineitem']['id'])]=float(productinfo['lineitem']['price'])/len(productinfo['lineitem']['composite_children'])
          print(str(productinfo['lineitem']['id']))
          continue
        if str(productinfo['lineitem']['composite_parent']) in Compositer.keys():
          productinfo['lineitem']['price'] = Compositer[ str( productinfo['lineitem']['composite_parent'] ) ]
          print(productinfo['lineitem']['price'])
        Report.append({'Date and Time':productinfo['datetime'],
                       'Product Info': productinfo['product_info']['name'],
                       'METRC Tag': productinfo['lineitem']['sku'],
                       'Total Weight':weight,
                       'Unit of Measure':typeweight,
                       'SubTotal':round(float(productinfo['lineitem']['price'])*float(productinfo['lineitem']['quantity']),2)
                       })
    return Report

In [10]:
TestProducts=GenerateProducts('2020-11-12')
FirstMETRC=GenerateReport(TestProducts)
df=pd.DataFrame(FirstMETRC).sort_values(by='Date and Time')
df.to_csv('./Federal2020_11_12.csv')

3.55  Shake
1 Chronic Creations | Wax   1G
 Juicy | Hemp Wraps
3.55  Shake
3.55  Shake
1 Greenleaf Concentrates | Budder   1G
1 Greenleaf Concentrates | Budder   1G
1 Greenleaf Concentrates | Budder   1G
1 Greenleaf Concentrates | Budder   1G
1 Canyon Cultivation | Suck IT   100mg
3.55  Shake
28.0  Shake
3.55  Shake
1   Single Pre Rolls
1   Single Pre Rolls
1   Single Pre Rolls
1   Single Pre Rolls
28.0  Shake
28.0  Shake
3.55  Dark Desire (I) | Eighth
 Pipe | Small
1 Gas Factory | Gummies   100mg
1 Gas Factory | Gummies   100mg
1 Altus | Gummies   100mg
1 Altus | Gummies   100mg
1   Kaviar | Joint – 1.5G
3.55  Shake
28.0  Shake
3.55  Cosmonaut (S) | Eighth
3.55  Dark Desire (I) | Eighth
3.55  Witches Weed (S) | Eighth
1 Greenleaf Concentrates | Budder   1G
1 Gas Factory | Gummies   100mg
1 Gas Factory | Gummies   100mg
 Juicy | Hemp Wraps
 Raw Cones   $5
1.05  Sundae Driver (H) | Gram
1.05  Secret Mints (H) | Gram
3.55  Shake
1 Kaviar | Infused Bud   1G
1   Single Pre Rolls
1 Kaviar |

Actually running the functions.

In [11]:
FirstMETRC=GenerateReport(TestProducts)

3.55  Shake
1 Chronic Creations | Wax   1G
1 Juicy | Hemp Wraps
3.55  Shake
3.55  Shake
1 Greenleaf Concentrates | Budder   1G
1 Greenleaf Concentrates | Budder   1G
1 Greenleaf Concentrates | Budder   1G
1 Greenleaf Concentrates | Budder   1G
1 Canyon Cultivation | Suck IT   100mg
3.55  Shake
28.0  Shake
3.55  Shake
1   Single Pre Rolls
1   Single Pre Rolls
1   Single Pre Rolls
1   Single Pre Rolls
28.0  Shake
28.0  Shake
3.55  Dark Desire (I) | Eighth
1 Pipe | Small
1 Gas Factory | Gummies   100mg
1 Gas Factory | Gummies   100mg
1 Altus | Gummies   100mg
1 Altus | Gummies   100mg
1   Kaviar | Joint – 1.5G
3.55  Shake
28.0  Shake
3.55  Cosmonaut (S) | Eighth
3.55  Dark Desire (I) | Eighth
3.55  Witches Weed (S) | Eighth
1 Greenleaf Concentrates | Budder   1G
1 Gas Factory | Gummies   100mg
1 Gas Factory | Gummies   100mg
1 Juicy | Hemp Wraps
1 Raw Cones   $5
1.05  Sundae Driver (H) | Gram
1.05  Secret Mints (H) | Gram
3.55  Shake
1 Kaviar | Infused Bud   1G
1   Single Pre Rolls
1 Kavi

In [12]:
df=pd.DataFrame(FirstMETRC).sort_values(by='Date and Time')

In [13]:
df.to_csv('./Federal2020_11_13.csv')

This lil guy right here is to be exported through the django hook.

In [14]:
df


,Date and Time,Product Info,METRC Tag,Total Weight,Unit of Measure,SubTotal
145,2020-11-11 21:54:32,Shake,1A4000B0000E0A1000004213,3.55,Grams,7.94
139,2020-11-12 08:02:03,Single Pre Rolls,1A4000B0000E0A1000002786,1.00,Grams,5.56
135,2020-11-12 08:23:28,Shake,1A4000B0000E0A1000004218,28.00,Grams,47.65
141,2020-11-12 08:46:22,Greenleaf Concentrates | Budder 1G,1A4000B0000E0A1000003650,1.00,Each,11.91
140,2020-11-12 08:46:22,Greenleaf Concentrates | Budder 1G,1A4000B0000E0A1000003655,1.00,Each,11.91
...,...,...,...,...,...,...
73,2020-11-12 22:02:23,Shake,1A4000B0000E0A1000004213,7.10,Grams,15.88
132,2020-11-12 22:02:29,Shake,1A4000B0000E0A1000004213,7.10,Grams,15.88
67,2020-11-12 22:02:31,Pre Rolled Joints 5 for $25,1A4000B0000E0A1000004171,5.00,Grams,19.86
1,2020-11-12 22:02:46,Chronic Creations | Wax 1G,1A4000B0000E0A1000003457,1.00,Each,19.86


Ideally I can use this to generate a daily to-do list, I wonder if I can get access to wheniwork chat and actually message the individuals who were logged in. That should be straightforward enough.

Ugh okay, so. 